In [2]:
import requests

In [3]:
def fetch_pages(category_title):
    url = "https://bots.snpedia.com/api.php"
    params = {
        "action" : "query",
        "list" : "categorymembers",
        "cmtitle" : category_title,
        "cmlimit" : "max",
        "format" : "json",
    }

    response = requests.get(url, params=params)
    data = response.json()

    pages = data["query"]["categorymembers"]

    return pages

In [ ]:
def fetch_page_content(page_id):
    url = "https://bots.snpedia.com/api.php"
    params = {
        "action" : "query",
        "prop" : "extracts",
        "pageids" : pageid,
        "format" : "json",
    }

    response = requests.get(url, params=params)
    data = response.json()

    content = data["query"]["pages"][str(page_id)]["extract"]

    return content